<h1><b>Restricted Boltzmann Machine και Deep Belief Network</b></h1>
<p align="justify">Στην συγκεκριμένη άσκηση θα μελετήσετε τον τρόπο λειτουργίας μιας <i>RBM (<a href="https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine">Restricted Boltzmann Machine</a>)</i> καθώς και των <i>DBN (<a href="https://en.wikipedia.org/wiki/Deep_belief_network">Deep Belief Network</a>)</i>, χρησιμοποιώντας το έτοιμο πρόγραμμα που σας δίνεται.Το συγκεκριμένο πρόγραμμα αξιοποιεί το <a href="https://en.wikipedia.org/wiki/MNIST_database">dataset του <i>MNIST</i></a>, όπου είναι μια μεγάλη βάση δεδομένων με χειρόγραφα ψηφία που χρησιμοποιείται συνήθως για την εκπαίδευση διαφόρων συστημάτων επεξεργασίας εικόνας. Για την άσκηση, θα πρέπει να χρησιμοποιήσετε το αρχείο <i>mnist_original.mat</i>, το οποίο είναι διαθέσιμο από <a href="https://www.kaggle.com/datasets/avnishnish/mnist-original?resource=download">εδώ</a>.</p>
<p align="justify">Μία αρκετά σημαντική εφαρμογή της <i>RBM</i> είναι η εξαγωγή χαρακτηριστικών (feature representation) από ένα dataset με σκοπό την αναπαράσταση της εισόδου (ορατοί νευρώνες) με ένα διάνυσμα μικρότερης διάστασης (κρυφοί νευρώνες). Στη συγκεκριμένη άσκηση θα συγκρίνετε την ακρίβεια ενός ταξινομητή ψηφίων με τη χρήση του αλγορίθμου <i>Logistic Regression</i>, όταν εκείνος δέχεται ως είσοδο το dataset (i) χωρίς να έχει υποστεί επεξεργασία από το <i>RBM</i>, (ii) αφου υποστεί επεξεργασία από το <i>RBM</i>, (iii) με τη χρήση <i>DBN</i>, δηλαδή δύο stacked <i>RBM</i>.</p>
<p align="justify"> Με βάση τον κώδικα που σας έχει δοθεί, καλείστε να απαντήσετε στα παρακάτω ερωτήματα:</p>
<ul>
<li>Να περιγράψετε σύντομα τον τρόπο λειτουργίας μιας <i>RBM</i>. Τι διαφορές έχει σε σχέση με μία <i> Μηχανή Boltzmann</i>;</li>
<li>Ποια είναι η λογική των <i>DBN</i> και σε τι προβλήματα τα αξιοποιούμε;</li>
<li>Να αναφέρετε τις βασικότερες εφαρμογές των <i>RBM</i> και <i>DBN</i>.</li>
<li>Εκτός από <i>RBM</i>, τι άλλα μοντέλα μπορούν να χρησιμοποιηθούν για να δημιουργήσουν <i>DBN</i>.</li>
<li>Συγκρίνετε τα αποτελέσματα της ταξινόμησης με τον αλγόριθμo <i>Logistic Regression</i> χωρίς τη χρήση <i>RBM</i> σε σχέση με τα αποτελέσματα της ταξινόμησης που έχει χρησιμοποιηθεί η <i>RBM</i> καθώς και με αυτή όπου χρησιμοποιούνται <i>RBM</i> και <i>DBN</i> για την εξαγωγή των χαρακτηριστικών. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;</li>
</ul>

In [1]:
#!/usr/bin/env python
# source: https://devdreamz.com/question/905929-stacking-rbms-to-create-deep-belief-network-in-sklearn

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

def norm(arr):
    arr = arr.astype(float)
    arr -= arr.min()
    arr /= arr.max()
    return arr

if __name__ == '__main__':

    # load MNIST data set
    mnist = loadmat("mnist-original.mat")
    X, Y = mnist["data"].T, mnist["label"][0]

    # normalize inputs to 0-1 range
    X = norm(X)

    # split into train, validation, and test data sets
    X_train, X_test, Y_train, Y_test = train_test_split(X,       Y,       test_size=10000, random_state=0)
    X_train, X_val,  Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=10000, random_state=0)

    # --------------------------------------------------------------------------------
    # set hyperparameters

    learning_rate = 0.02 
    total_units   =  800 
    total_epochs  =   50
    batch_size    =  128

    C = 100. # optimum for benchmark model according to sklearn docs: https://scikit-learn.org/stable/auto_examples/neural_networks/plot_rbm_logistic_classification.html#sphx-glr-auto-examples-neural-networks-plot-rbm-logistic-classification-py)

    # --------------------------------------------------------------------------------
    # construct models

    # RBM
    rbm = BernoulliRBM(n_components=total_units, learning_rate=learning_rate, batch_size=batch_size, n_iter=total_epochs, verbose=1)

    # "output layer"
    logistic = LogisticRegression(C=C, solver='lbfgs', multi_class='multinomial', max_iter=200, verbose=1)

    models = []
    models.append(Pipeline(steps=[('logistic', clone(logistic))]))                                              # base model / benchmark
    models.append(Pipeline(steps=[('rbm1', clone(rbm)), ('logistic', clone(logistic))]))                        # single RBM
    models.append(Pipeline(steps=[('rbm1', clone(rbm)), ('rbm2', clone(rbm)), ('logistic', clone(logistic))]))  # RBM stack / DBN

    # --------------------------------------------------------------------------------
    # train and evaluate models

    for model in models:
        # train
        model.fit(X_train, Y_train)

        # evaluate using validation set
        print("Model performance:\n%s\n" % (
            classification_report(Y_val, model.predict(X_val))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\rigas\anaconda3\envs\DSMLEnv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96       995
         1.0       0.96      0.98      0.97      1121
         2.0       0.90      0.90      0.90      1015
         3.0       0.90      0.88      0.89      1033
         4.0       0.93      0.92      0.92       976
         5.0       0.90      0.88      0.89       884
         6.0       0.94      0.94      0.94       999
         7.0       0.92      0.93      0.92      1034
         8.0       0.88      0.86      0.87       923
         9.0       0.89      0.90      0.90      1020

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000


[BernoulliRBM] Iteration 1, pseudo-likelihood = -142.32, time = 5.72s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -118.06, time = 8.31s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -106.05, time = 8.50s
[BernoulliR

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\rigas\anaconda3\envs\DSMLEnv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.3s finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       995
         1.0       0.98      0.99      0.99      1121
         2.0       0.97      0.97      0.97      1015
         3.0       0.97      0.97      0.97      1033
         4.0       0.98      0.97      0.97       976
         5.0       0.97      0.97      0.97       884
         6.0       0.98      0.98      0.98       999
         7.0       0.97      0.98      0.97      1034
         8.0       0.96      0.95      0.96       923
         9.0       0.96      0.96      0.96      1020

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


[BernoulliRBM] Iteration 1, pseudo-likelihood = -140.06, time = 5.81s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -117.58, time = 8.32s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -104.38, time = 8.34s
[BernoulliR

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\rigas\anaconda3\envs\DSMLEnv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       995
         1.0       0.99      0.99      0.99      1121
         2.0       0.98      0.98      0.98      1015
         3.0       0.98      0.97      0.98      1033
         4.0       0.98      0.97      0.97       976
         5.0       0.98      0.97      0.97       884
         6.0       0.99      0.99      0.99       999
         7.0       0.97      0.98      0.98      1034
         8.0       0.96      0.97      0.97       923
         9.0       0.96      0.97      0.96      1020

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000




### Απαντήσεις

- <font color='#486393'>Πριν γίνει αναφορά στις Restricted Boltzmann Machines (RBMs), θα πρέπει πρώτα να συζητηθούν συνοπτικά οι καθιερωμένες Μηχανές Boltzmann (Boltzmann Machines - BMs), αφού αυτές αποτελούν τη γενίκευση των RBMs. Μια Μηχανή Boltzmann αποτελεί ένα είδος ρηχού νευρωνικού δικτύου το οποίο αποτελείται από ένα σύνολο φανερών νευρώνων (φανερό επίπεδο) και ένα σύνολο κρυφών νευρώνων (κρυφό επίπεδο) και μπορεί να απεικονιστεί ως ένας διμερής γράφος. Επιπλέον, χαρακτηρίζεται από συμμετρικές συνάψεις, δηλαδή μεταξύ δύο νευρώνων $i$, $j$ υπάρχουν και η σύναψη $i \to j$, αλλά και η σύναψη $j \to i$, με ίσα συναπτικά βάρη, $w_{ij} = w_{ji}$. Η Μηχανή Boltzmann χρησιμοποιείται για την εκμάθηση χαρακτηριστικών μιας άγνωστης κατανομής πιθανότητας, αξιοποιώντας διαθέσιμα δείγματα από την κατανομή αυτή. Γενικά, η διαδικασία αυτή είναι περίπλοκη και χρονοβόρα, μπορεί όμως να απλοποιηθεί και να επιταχυνθεί εάν κανείς επιβάλλει ορισμένους περιορισμούς στην τοπολογία του δικτύου, εξ ου και η έννοια της "Restricted" Boltzmann Machine.</font>

<font color='#486393'>Μια RBM αποτελεί υποπερίπτωση Μηχανής Boltzmann, στην οποία δεν υπάρχουν συνάψεις μεταξύ νευρώνων που ανήκουν στο ίδιο επίπεδο. Με άλλα λόγια, οι συνάψεις με εν γένει μη μηδενικά συναπτικά βάρη είναι μόνο αυτές μεταξύ ενός κρυφού και ενός φανερού νευρώνα. Στην παρακάτω εικόνα φαίνονται δύο μηχανές Boltzmann που αποτελούνται από ένα φανερό επίπεδο 3 νευρώνων και ένα κρυφό επίπεδο 2 νευρώνων. Η βασική τους διαφορά, η οποία καθιστά τη μηχανή στα δεξιά ως RBM, έγκειται στον περιορισμό αυτό σχετικά με τις συνάψεις.</font><br>

<center><img src="RBMImage.png" width="600" /></center><br>

<font color='#486393'>Σε ό,τι έχει να κάνει με την αρχή λειτουργίας τους, για κάθε στοιχείο $\mathbf{x}_v$ του δείγματος, η RBM προσπαθεί σε βάθος $E$ εποχών μιας επαναληπτικής διαδικασίας να μάθει να ανακατασκευάζει το στοιχείο αυτό. Αυτός είναι και ο λόγος για τον οποίο μιλάμε για παραγωγική (generative) διαδικασία. Με τον τρόπο αυτό, η RBM εκπαιδεύεται ώστε το ορατό της επίπεδο να πραγματοποιεί δειγματοληψία από μια κατανομή Gibbs, η οποία όμως προσεγγίζει όσο το δυνατό περισσότερο την άγνωστη κατανομή από την οποία προκύπτουν τα δειγματικά σημεία του συνόλου εκπαίδευσης.</font>

<font color='#486393'>Στην πράξη, η διαδικασία εκπαίδευσης της RBM ξεκινά με την αρχικοποίηση όλων των μη μηδενικών συναπτικών βαρών σε τυχαίες τιμές. Κάθε νευρώνας μπορεί να βρίσκεται σε μία από δύο επιτρεπτές καταστάσεις, έστω 0 και 1 (εξ ου και ο χαρακτηρισμός «Bernoulli» RBM στον παραπάνω κώδικα) και ενεργοποιείται μέσω της σιγμοειδούς συνάρτησης. Στην αρχή κάθε εποχής, $e$, με $e \in \left\{0,\dots,E\right\}$, ο αλγόριθμος λαμβάνει ως είσοδο των ορατών νευρώνων κάθε στοιχείο $\mathbf{x}_v$ του δείγματος και εν παραλλήλω το προωθεί στους κρυφούς νευρώνες, όπου υπολογίζεται η ενεργοποίησή τους (forward pass). Από εκεί, πραγματοποιείται δειγματοληψία κατά Gibbs και οι τιμές που παράγονται προωθούνται εν παραλλήλω στους ορατούς νευρώνες (μέσω των ίδιων συναπτικών βαρών με πριν λόγω συμμετρίας), όπου υπολογίζεται η ενεργοποίησή τους (backward pass), $\tilde{\mathbf{x}}_v$. Με το πέρας της εποχής, τα συναπτικά βάρη ανανεώνονται έτσι, ώστε να μεγιστοποιείται ο λογάριθμος της πιθανοφάνειας των ανεξάρτητων μεταξύ τους δειγματικών σημείων (το οποίο ισοδυναμεί με την ελαχιστοποίηση των διαφοροποιήσεων ανάμεσα σε $\mathbf{x}_v$ και $\tilde{\mathbf{x}}_v$).</font>

<font color='#486393'>Ένα σημείο στο οποίο αξίζει κανείς να σταθεί είναι το γεγονός πως η δειγματοληψία από την κατανομή Gibbs μπορεί να πραγματοποιηθεί γρήγορα (μέσω της προαναφερθείσας παραλληλοποίησης) ακριβώς επειδή οι κρυφοί νευρώνες είναι μεταξύ τους ανεξάρτητοι δεδομένων των ορατών και αντιστρόφως. Αυτή είναι η αφορμή για τη διαφοροποίηση των RBM από τις καθιερωμένες Μηχανές Boltzmann. Παρ' όλα αυτά, η δειγματοληψία κατά Gibbs κατά το backward pass δε μπορεί να γίνει αμερόληπτα εάν δεν έχει επέλθει ισορροπία, εάν δηλαδή το δίκτυο δεν έχει μεταβεί στην κατάσταση που περιγράφεται από την αναλλοίωτη κατανομή Markov του. Για το σκοπό αυτό επιστρετεύονται διάφοροι αλγόριθμοι, όπως είναι ο Contrastive Divergence (CD) που συζητήθηκε στο μάθημα. Μέσω αυτών, παρότι δεν επιτυγχάνεται η ελαχιστοποίηση της πιθανοφάνειας, επιτυγχάνεται η ελαχιστοποίηση της απόκλισης Kullback-Leibler μεταξύ κατανομής Gibbs και άγνωστης κατανομής, δίνοντας τελικά ικανοποιητικά αποτελέσματα.</font>

<!--<font color='#486393'>αποτελείται από ένα σύνολο βημάτων, $t$, με $t \in \left\{0,\dots,T\right\}$. Κάθε βήμα αποτελείται από διαδοχικές παραγωγές τιμών: πρώτα από όλους τους ορατούς νευρώνες, οι οποίες προωθούνται στους κρυφούς νευρώνες για να παράξουν στοιχεία $\mathbf{x}_h^{(t)}\left(e\right)$, και κατόπιν από τους κρυφούς νευρώνες οι οποίες προωθούνται στους ορατούς νευρώνες για να παράξουν στοιχεία $\mathbf{x}_v^{(t+1)}\left(e\right)$. Προφανώς, η αρχικοποίηση γίνεται ως $\mathbf{x}_v^{(0)}\left(0\right) = \mathbf{x}_v$. Κάθε νευρώνας μπορεί να βρίσκεται σε μία από δύο επιτρεπτές καταστάσεις, έστω 0 και 1, εξ ου και ο χαρακτηρισμός «Bernoulli» RBM στον παραπάνω κώδικα, και ενεργοποιείται μέσω της σιγμοειδούς συνάρτησης. Μετά από $T$ βήματα, οι διαφοροποιήσεις ως προς τα συναπτικά βάρη της κατάστασης κατά την αρχή της εποχής, $\mathbf{x}_v^{(0)}\left(e\right)$, και κατά το τέλος της εποχής, $\mathbf{x}_v^{(T)}\left(e\right)$, ελαχιστοποιούνται, προκειμένου να μεγιστοποιηθεί ο λογάριθμος της πιθανοφάνειας όλων των ανεξάρτητων δειγματικών σημείων. Έτσι, με το πέρας $E$ εποχών, επιτυγχάνεται η δημιουργία καταστάσεων $\mathbf{x}_v^{(T)}\left(E\right)$ οι οποίες αντιστοιχούν σε μια κατανομή Gibbs, η οποία με τη σειρά της έχει μικρή απόκλιση Kullback-Leibler από την άγνωστη κατανομή από την οποία προκύπτουν τα δειγματικά σημεία $\mathbf{x}_v$. Έτσι, η RBM επιτρέπει στην ουσία τη δημιουργία μιας κατανομής η οποία να προσεγγίζει όσο το δυνατό περισσότερο την άγνωστη κατανομή. Στη διαδικασία αυτή, η μη διασύνδεση μεταξύ νευρώνων του ίδιου επιπέδου επιταχύνει την παραγωγή των επιθυμητών στοιχείων και αποτελεί την αφορμή για την επιβολή του επιπλέον περιορισμού που διαχωρίζει τις RBM από τις υπόλοιπες Μηχανές Boltzmann.</font>-->

- <font color='#486393'>Τα Deep Belief Networks (DBNs) αποτελούν στη βάση τους μια σύνθεση απλούστερων δικτύων, όπως είναι για παράδειγμα οι RBMs που συζητώνται στο παρόν Notebook, ή δομές όπως οι autoencoders (οι οποίοι είναι και ο λόγος για τον οποίο οι RBMs δε χρησιμοποιούνται πλέον συχνά). Η αρχιτεκτονική τους είναι τέτοια, ώστε το κρυφό επίπεδο κάθε απλούστερης δομής να αποτελεί το ορατό επίπεδο της επόμενης.</font>

<font color='#486393'>Χρησιμοποιούνται σε προβλήματα που χρησιμοποιούνται και οι RBMs, τα οποία αφορούν κυρίως την εξαγωγή χαρακτηριστικών από δεδομένα μέσω μη επιβλεπόμενης μάθησης, αφού επιτρέπουν την αναπαράστασή τους από μια αρχική μορφή σε μορφές υψηλότερου επιπέδου (higher level representations). Φυσικά, μπορούν να εκπαιδευτούν και περαιτέρω μέσω επίβλεψης και να αξιοποιηθούν σε προβλήματα ταξινόμησης, ή να αποτελέσουν το πρώτο κομμάτι μιας βαθύτερης αρχιτεκτονικής.</font>

<font color='#486393'>Αυτό που διαφοροποιεί τα DBNs από τις RBMs είναι πως είναι πιο αποδοτικά στην επίλυση των προβλημάτων αυτών, καθώς το αυξημένο τους βάθος επιδρά στην απόδοσή τους με τον τρόπο που επιδρά και στα γνωστά βαθιά δίκτυα, επιτρέποντάς τους δηλαδή να εκπαιδευτούν αποτυπώνοντας πιο σύνθετες αλληλεπιδράσεις. Μάλιστα, ένα σημαντικό τους προτέρημα είναι πως η δυνατότητά τους να αφομοιώνουν πολυπλοκότερες δομές δε μεταφράζεται σε αυξημένη πολυπλοκότητα στον τρόπο εκπαίδευσής τους. Αντιθέτως, μπορούν να εκπαιδευτούν «greedily», δηλαδή εφαρμόζοντας αλγορίθμους εκπαίδευσης όπως ο CD σε κάθε απλούστερο δίκτυο ξεχωριστά και θεωρώντας την έξοδο στην οποία συγκλίνει ως είσοδο του επόμενου.</font>

- <font color='#486393'>Πέραν της εξαγωγής χαρακτηριστικών και της ταξινόμησης, οι οποίες αναφέρθηκαν παραπάνω, ορισμένες επιπλέον εφαρμογές όπου αξιοποιούνται τα DBNs και οι RBMs είναι οι ακόλουθες:</font>

<font color='#486393'>[1] Διαστατική μείωση, επιλέγοντας εξόδους μικρότερου μεγέθους σε σχέση με την είσοδο, δηλαδή το σύνολο των δειγματικών σημείων.</font>

<font color='#486393'>[2] Συστήματα σύστασης (Recommender systems), όπου δεδομένων ορισμένων δεδομένων εισόδου, π.χ. ταινίες, αναγνωρίζουν κάποια κρυφά χαρακτηριστικά όπως το είδος ταινίων που προτιμάται συχνότερα και βάσει αυτού επιστρέφουν άλλες ταινίες αντίστοιχου είδους.</font>

<font color='#486393'>[3] Θεματική μοντελοποίηση (topic modelling), κυρίως στην περιοχή της επεξεργασίας φυσικής γλώσσας, η οποία εμφανίζει συγγένειες με την έννοια της συσταδοποίησης δεδομένων.</font>

<font color='#486393'>[4] Ανακατασκευή εικόνας (image reconstruction), όπου θολές εικόνες ενισχύονται ή κρυφά σημεία σε φωτογραφίες (π.χ. το πίσω μέρος ενός σπιτιού για το οποίο φαίνεται μόνο η πρόσοψη) ανακατασκευάζονται.</font>

<font color='#486393'>[5] Παραγωγή βίντεο, αξιοποιώντας τη δυνατότητα δειγματοληψίας για διαφορετικά καρέ (frames).</font>

<font color='#486393'>[6] Στη Φυσική, για τη μελέτη κβαντικών συστημάτων όπου μετρήσεις στις παρατηρούμενες καταστάσεις τους μπορούν να χρησιμοποιηθούν για την ανακατασκευή της κυματοσυνάρτησης που τα περιγράφει.</font>

<font color='#486393'>[7] Στην Ιατρική, σε εγκεφαλογραφήματα (βλ. [εδώ](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5967386/)).</font>

<font color='#486393'>Σε κάθε περίπτωση, πρέπει να έχει κανείς υπ' όψιν πως τα RBMs και τα DBNs που κατασκευάζονται από αυτά έχουν αντικατασταθεί για τις εφαρμογές αυτές από συστήματα όπως τα GANs (Generative Adversarial Networks) ή οι προαναφερθέντες autoencoders.

- <font color='#486393'>Όπως αναφέρθηκε και στην απάντηση του δεύτερου ερωτήματος, οι RBMs δεν έχουν το μονοπώλιο στην κατασκευή DBNs. Άλλες δομές που μπορούν να αξιοποιηθούν για την κατασκευή DBNs είναι οι autoencoders (βλ. [εδώ](https://ieeexplore.ieee.org/abstract/document/7882660)), τα κρυφά Μαρκοβιανά μοντέλα (Hidden Markov Models - HMMs - βλ. [εδώ](https://ieeexplore.ieee.org/document/6707732)), ή τα GANs (Generative Adversarial Networks - βλ. [εδώ](https://arxiv.org/abs/1909.06134)). </font>

- <font color='#486393'>Σε όλα τα αποτελέσματα που φαίνονται στο classification report του κώδικα, η ακρίβεια (Accuracy) είναι πλήρως εναρμονισμένη με τις macro- και micro-averaged εκδοχές των Precision, Recall και F1-Score, επομένως τα μοντέλα δεν έχουν συγκεκριμένες προτιμήσεις σε τύπους λαθών και άρα η ακρίβεια αυτή καθ' αυτή επαρκεί για την αξιολόγηση των μοντέλων. Βάσει αυτών, τα αποτελέσματα για κάθε μοντέλο συνοψίζονται στον ακόλουθο πίνακα.</font>

| Μοντέλο | Ακρίβεια |
| :---: | :---: |
| Λογιστική παλινδρόμηση χωρίς RBM | **92%** |
| Λογιστική παλινδρόμηση με RBM | **97%** |
| Λογιστική παλινδρόμηση με DBN από 2 RBM | **98%** |

<font color='#486393'>Επειδή τα μοντέλα εκπαιδεύτηκαν για πολλές εποχές, σε πολύ μεγάλο όγκο δεδομένων, τα οποία έχουν μια σχετική απλότητα (αποτελούν το «Hello world» πρόβλημα για την αναγνώριση εικόνας), όλα τους εμφανίζουν σχετικά καλή απόδοση. Παρ' όλα αυτά, η επίδραση της εξαγωγής χαρακτηριστικών των RBMs είναι εμφανής, αφού ένα RBM στην είσοδο του μοντέλου λογιστικής παλινδρόμησης οδηγεί σε αύξηση της απόδοσης σε ακόμα υψηλότερα ποσοστά. Σημειώνεται πως, παρότι η αύξηση αυτή είναι μόλις +5%, έχει σημασία το γεγονός ότι αποτελεί αύξηση σε ένα ήδη υψηλό ποσοστό (92%). Σε ό,τι αφορά το τρίτο μοντέλο, τα αποτελέσματα επίσης επαληθεύουν την ανάλυση που πραγματοποιήθηκε στα προηγούμενα ερωτήματα, αφού το τελικό μοντέλο εμφανίζει τη μέγιστη ακρίβεια, επιτυγχάνοντας να αυξήσει την ήδη πολύ υψηλή ακρίβεια του μοντέλου λογιστικής παλινδρόμησης με 1 RBM στην είσοδό του (97%). Σίγουρα, η επίδραση που έχει η προσθήκη του/των RBM(s) στο συνολικό χρόνο εκτέλεσης δεν είναι αμελητέα, όμως η αύξηση της συνολικής ακρίβειας στην οποία οδηγούν σίγουρα έχει πολύ υψηλότερη βαρύτητα.</font>